In [275]:
import pandas as pd
import numpy as np
import datetime as dt
import re

In [475]:
admissions = pd.read_csv("../Data/ADMISSIONS.csv")
icustays = pd.read_csv("../Data/ICUSTAYS.csv")
patients = pd.read_csv("../Data/PATIENTS.csv")
imputevents_mv = pd.read_csv("../Data/INPUTEVENTS_MV.csv")
diagnoses_icd = pd.read_csv("../Data/DIAGNOSES_ICD.csv")

In [120]:
def time_in_between(start, end, day_or_year = 'day'):
    start = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    end = dt.datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
    delta = end - start
    if day_or_year == 'day':
        return delta.days + delta.seconds / 86400
    else:
        return (delta.days + delta.seconds / 86400) / 365

In [467]:
def one_hot_encoding(variable, data):
    ohc = pd.get_dummies(data[variable].apply(pd.Series).stack()).sum(level=0)
    data = pd.concat([data, ohc.iloc[:, :-1]], axis=1)
    return data

In [328]:
def translate_icd(icd):
    if bool(re.search(r'[VE]', icd)):
        return 'external_cause'
    icd = eval(re.findall(r'[^0]\d*', icd)[0])
    if icd <= 139:
        return 'parasitic_disease'
    if icd <= 239:
        return 'neoplasms'
    if icd <= 279:
        return 'metabolic_disease'
    if icd <= 289:
        return 'blood_disease'
    if icd <= 319:
        return 'mental_disorder'
    if icd <= 389:
        return 'nervous_system'
    if icd <= 459:
        return 'circulatory_system'
    if icd <= 519:
        return 'respiratory_system'
    if icd <= 579:
        return 'digestive_system'
    if icd <= 629:
        return 'genitourinary_system'
    if icd <= 679:
        return 'pregnancy'
    if icd <= 709:
        return 'skin_disease'
    if icd <= 739:
        return 'musculoskeletal_system'
    if icd <= 759:
        return 'congenital'
    if icd <= 779:
        return 'perinatal period'
    if icd <= 799:
        return 'ill_defined_condition'
    if icd <= 999:
        return 'injury'
    return 'unknown'

In [476]:
data = pd.merge(admissions, icustays, on = ['HADM_ID', 'SUBJECT_ID'], how = 'left')
data = data[~data.OUTTIME.isnull()]
data = data[data.DEATHTIME.isnull()]
data['TOTAL_LOS'] = [time_in_between(data.ADMITTIME.iloc[i], data.DISCHTIME.iloc[i], day_or_year = 'day') for i in range(len(data))]
data['LOS_POST_ICU'] = [time_in_between(data.OUTTIME.iloc[i], data.DISCHTIME.iloc[i], day_or_year = 'day') for i in range(len(data))]
data['LOS_POST_ICU'] = [0 if day < 0 else day for day in data['LOS_POST_ICU']]

data = pd.merge(data, patients, on = 'SUBJECT_ID', how = 'left')
data['AGE'] = [time_in_between(data.DOB.iloc[i], data.ADMITTIME.iloc[i], day_or_year = 'year') for i in range(len(data))]
data['AGE'] = [age - 300 if age > 300 else age for age in data['AGE']]

imputevents_mv = imputevents_mv[['SUBJECT_ID', 'PATIENTWEIGHT']]
imputevents_mv['SID'] = imputevents_mv['SUBJECT_ID']
imputevents_mv = imputevents_mv.groupby(['SID']).mean()
data = pd.merge(data, imputevents_mv, on = 'SUBJECT_ID', how = 'left')

diagnoses_icd = diagnoses_icd.dropna()
diagnoses_icd['DIAGNOSES'] = [translate_icd(icd) for icd in diagnoses_icd.ICD9_CODE.str.slice(start=0, stop=3, step=1)]
diagnoses_icd = diagnoses_icd.groupby(['SUBJECT_ID', 'HADM_ID'])['DIAGNOSES'].apply(','.join).reset_index()
diagnoses_icd['DIAGNOSES'] = [d.split(',') for d in diagnoses_icd['DIAGNOSES']]
diagnoses_ohc = pd.get_dummies(diagnoses_icd['DIAGNOSES'].apply(pd.Series).stack()).sum(level=0)
diagnoses_icd = pd.concat([diagnoses_icd, diagnoses_ohc], axis=1)
data = pd.merge(data, diagnoses_icd, on = ['HADM_ID', 'SUBJECT_ID'], how = 'left')

data['LANGUAGE'][data['LANGUAGE'].isnull()] = 'nan'
data['LANGUAGE'] = ['English' if 'ENGL' in lang 
                 else 'Unknown' if 'nan' in lang 
                 else 'Non-English' for lang in data['LANGUAGE']]
data['RELIGION'][data['RELIGION'].isnull()] = 'UNOBTAINABLE'
data['RELIGION'] = ['Unobtainable' if 'UNOBTAINABLE' in r
                 else 'Not Specified' if 'NOT SPECIFIED' in r 
                 else 'Religious' for r in data['RELIGION']]
data['MARITAL_STATUS'][data['MARITAL_STATUS'].isnull()] = 'UNKNOWN (DEFAULT)'
data['ETHNICITY'].replace(regex = r'^WHITE\D*', value = 'White', inplace=True)
data['ETHNICITY'].replace(regex = r'^BLACK\D*', value = 'Black', inplace=True)
data['ETHNICITY'].replace(regex = r'^ASIAN\D*', value = 'Asian', inplace=True)
data['ETHNICITY'].replace(regex = r'^HISPANIC\D*', value = 'Hispanic/Latino', inplace=True)
data['ETHNICITY'] = [e if 'White' in e
                  else e if 'Black' in e 
                  else e if 'Asian' in e
                  else e if 'Hispanic/Latino' in e
                  else 'Other/Unknown' for e in data['ETHNICITY']]

data = one_hot_encoding('ADMISSION_TYPE', data)
data = one_hot_encoding('ADMISSION_LOCATION', data)
data = one_hot_encoding('INSURANCE', data)
data = one_hot_encoding('LANGUAGE', data)
data = one_hot_encoding('RELIGION', data)
data = one_hot_encoding('MARITAL_STATUS', data)
data = one_hot_encoding('ETHNICITY', data)
data = one_hot_encoding('GENDER', data)


data = data[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 
             'blood_disease', 'circulatory_system', 'congenital', 'digestive_system', 'external_cause', 
             'genitourinary_system', 'ill_defined_condition', 'injury', 'mental_disorder', 'metabolic_disease', 
             'musculoskeletal_system', 'neoplasms', 'nervous_system', 'parasitic_disease', 'perinatal period', 
             'pregnancy', 'respiratory_system', 'skin_disease', 'INSURANCE', 'LANGUAGE', 'RELIGION', 
             'MARITAL_STATUS', 'ETHNICITY', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 
             'INTIME', 'OUTTIME', 'LOS','GENDER', 'AGE', 'PATIENTWEIGHT', 'ELECTIVE', 'EMERGENCY', 'NEWBORN', 
             '** INFO NOT AVAILABLE **', 'CLINIC REFERRAL/PREMATURE', 'EMERGENCY ROOM ADMIT', 'HMO REFERRAL/SICK',
             'PHYS REFERRAL/NORMAL DELI', 'TRANSFER FROM HOSP/EXTRAM', 'TRANSFER FROM OTHER HEALT', 
             'TRANSFER FROM SKILLED NUR', 'Government', 'Medicaid', 'Medicare', 'Private', 'English', 'Non-English',
             'Not Specified', 'Religious', 'DIVORCED', 'LIFE PARTNER', 'MARRIED', 'SEPARATED', 'SINGLE', 'UNKNOWN (DEFAULT)', 
             'Asian', 'Black', 'Hispanic/Latino', 'Other/Unknown', 'F', 'TOTAL_LOS', 'LOS_POST_ICU']]

/Users/mike/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mike/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mike/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [491]:
post_icu_df = data[['ADMISSION_TYPE', 'ADMISSION_LOCATION', 'blood_disease', 'circulatory_system', 'congenital', 
              'digestive_system', 'external_cause', 'genitourinary_system', 'ill_defined_condition', 'injury', 
              'mental_disorder', 'metabolic_disease', 'musculoskeletal_system', 'neoplasms', 'nervous_system', 
              'parasitic_disease', 'perinatal period', 'pregnancy', 'respiratory_system', 'skin_disease','INSURANCE', 
              'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 
              'FIRST_WARDID', 'LAST_WARDID', 'LOS','GENDER', 'AGE', 'PATIENTWEIGHT', 'LOS_POST_ICU']]

post_icu_df.columns = ['ADMISSION_TYPE', 'ADMISSION_LOCATION', 'blood_disease', 'circulatory_system', 'congenital', 
              'digestive_system', 'external_cause', 'genitourinary_system', 'ill_defined_condition', 'injury', 
              'mental_disorder', 'metabolic_disease', 'musculoskeletal_system', 'neoplasms', 'nervous_system', 
              'parasitic_disease', 'perinatal period', 'pregnancy', 'respiratory_system', 'skin_disease','INSURANCE', 
              'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 
              'FIRST_WARDID', 'LAST_WARDID', 'LOS_ICU','GENDER', 'AGE', 'PATIENTWEIGHT', 'LOS_POST_ICU']

los_df = data[['ADMISSION_TYPE', 'ADMISSION_LOCATION', 'blood_disease', 'circulatory_system', 'congenital', 
              'digestive_system', 'external_cause', 'genitourinary_system', 'ill_defined_condition', 'injury', 
              'mental_disorder', 'metabolic_disease', 'musculoskeletal_system', 'neoplasms', 'nervous_system', 
              'parasitic_disease', 'perinatal period', 'pregnancy', 'respiratory_system', 'skin_disease','INSURANCE', 
              'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY','GENDER', 'AGE', 'PATIENTWEIGHT', 'TOTAL_LOS']]

In [482]:
los_df_ohc = data[['ELECTIVE', 'EMERGENCY', 'NEWBORN', '** INFO NOT AVAILABLE **', 'CLINIC REFERRAL/PREMATURE', 
                   'EMERGENCY ROOM ADMIT', 'HMO REFERRAL/SICK', 'PHYS REFERRAL/NORMAL DELI', 'TRANSFER FROM HOSP/EXTRAM',
                   'TRANSFER FROM OTHER HEALT', 'TRANSFER FROM SKILLED NUR',  'blood_disease', 'circulatory_system',
                   'congenital', 'digestive_system', 'external_cause', 'genitourinary_system', 'ill_defined_condition', 
                   'injury', 'mental_disorder', 'metabolic_disease', 'musculoskeletal_system', 'neoplasms', 'nervous_system', 
                   'parasitic_disease', 'perinatal period', 'pregnancy', 'respiratory_system', 'skin_disease',
                   'Government', 'Medicaid', 'Medicare', 'Private', 'English', 'Non-English', 'Not Specified', 
                   'Religious', 'DIVORCED', 'LIFE PARTNER', 'MARRIED', 'SEPARATED', 'SINGLE', 'UNKNOWN (DEFAULT)', 'Asian', 
                   'Black', 'Hispanic/Latino', 'Other/Unknown', 'F', 'AGE', 'PATIENTWEIGHT', 'TOTAL_LOS']]

In [493]:
post_icu_df.to_csv("Model_Data_V1.csv")
los_df.to_csv("Model_Data_V3.csv")
los_df_ohc.to_csv("Model_Data_V3_ohc.csv")